In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Grounding with Vertex AI Search

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgrounding%2Fgrounding_with_vais.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/grounding/grounding_with_vais.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/53/X_logo_2023_original.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
| Author(s) |  [Diem Vu](https://github.com/diemtvu/) [Zhen Hu](https://github.com/undertwig/) |

## Overview

This notebook demonstrates how to use Vertex AI Search for grounding Vertex LLMs.  For more general information on grounding, see [Getting Started with Grounding with Gemini in Vertex AI](./intro-grounding-gemini.ipynb).

In this tutorial, we will cover:

* How to create a Vertex AI Search datastore with your data.
* Provide an example LLM request that uses this data for grounding.

## Get started

### Install Vertex AI SDK and other required packages

In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform google-cloud-discoveryengine

# install Google Cloud storage client library. Needed to import document to data store.
%pip install -U google-cloud-storage

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

[link text](https://)<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. In Colab or Colab Enterprise, you might see an error message that says "Your session crashed for an unknown reason." This is expected. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

# You also need to choose a region for your Vertex AI Search datastore.
# Vertex AI Search is a multi-region service, supporting us, eu, and global regions.
VAIS_LOCATION = "global"  # @param {type: "string"}

## Create Vertex AI Search engine

We will create a Vertex AI Search engine index by ingesting files from Google Cloud Storage (GCS) using the SDK. To explore additional features and UI-based instructions, see the [Vertex AI Search documentation](https://cloud.google.com/enterprise-search).

This involves 2 steps:

1. Create a datastore (corpus) and import documents: For this demo, we will use the PDF files available at `gs://cloud-samples-data/gen-app-builder/search/alphabet-sec-filings`. You can try with your own data.

2. Create `SEARCH_TIER_ENTERPRISE` engine with `SEARCH_ADD_ON_LLM` feature on top of the data store. This is essential to get high grounded generation quality.

In [ ]:
# @title Initialize the VAIS clients

import json
import time

from google.api_core.client_options import ClientOptions
from google.cloud import storage
import google.cloud.discoveryengine_v1 as vais

client_options = (
    ClientOptions(api_endpoint=f"{VAIS_LOCATION}-discoveryengine.googleapis.com")
    if VAIS_LOCATION != "global"
    else None
)
data_store_service_client = vais.DataStoreServiceClient(
    client_options=client_options, credentials=scoped_credentials
)
document_service_client = vais.DocumentServiceClient(
    client_options=client_options, credentials=scoped_credentials
)
engine_client = vais.EngineServiceClient(
    client_options=client_options
)

def wait_for_operation_finish(operation):
    while not operation.done:
        time.sleep(2)  # sleep 2 seconds

In [ ]:
# @title Create data store

DATA_STORE_ID = "your-data-store-id"  # @param {type:"string"}

# Create a generic search data store with unstrucuted data content.
data_store = vais.DataStore(
    display_name="Data Store for Vertex LLM Grounding demo",
    industry_vertical="GENERIC",
    solution_types=["SOLUTION_TYPE_SEARCH"],
    content_config="CONTENT_REQUIRED",
)

create_data_store_request = vais.CreateDataStoreRequest(
    parent=f"projects/{PROJECT_ID}/locations/{VAIS_LOCATION}/collections/default_collection",
    data_store=data_store,
    data_store_id=DATA_STORE_ID,
)

# The api returns long running operation as response.
create_data_store_operation = data_store_service_client.create_data_store(
    create_data_store_request
)
print(
    f"Waiting for create datastore operation to complete: {create_data_store_operation.operation.name}"
)
created_data_store = create_data_store_operation.result()
data_store_name = created_data_store.name
print(f"Data store {data_store_name} is created.")

In [ ]:
# @title Ingest documents

BUCKET_NAME = "cloud-samples-data"  # @param {type:"string"}
DATA_FOLDER = "'gen-app-builder/search/alphabet-sec-filings" # @param {type:"string"}

storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)

i = 1
for blob in bucket.list_blobs(prefix=DATA_FOLDER):
    metadata = {}
    document = vais.Document(
        content=vais.Document.Content(
            mime_type="application/pdf",
            uri=f"gs://{BUCKET_NAME}/{blob.name}",
        ),
        json_data=json.dumps(metadata),
    )

    create_document_request = vais.CreateDocumentRequest(
        parent=f"{data_store_name}/branches/0",
        document_id=f"document-{id:02d}",
        document=document,
    )
    i += 1
    created_document = document_service_client.create_document(create_document_request)
    document_name = created_document.name
    created_document_names.append(document_name)
print(f'{i} documents imported') 

In [ ]:
# @title [Optional] Verify documents has been imported.

for doc in document_service_client.list_documents(parent=f"{data_store_name}/branches/0"):
  print (doc.name)

In [ ]:
# @title Create engine

engine_id = f'{DATA_STORE_ID}_engine'
create_engine_request = vais.CreateEngineRequest(
    parent=f"projects/{PROJECT_ID}/locations/global/collections/default_collection",
    engine=vais.Engine(
        display_name="Engine for Vertex LLM Grounding demo",
        solution_type=vais.SolutionType.SOLUTION_TYPE_SEARCH,
        search_engine_config=vais.Engine.SearchEngineConfig(
            search_tier=vais.SearchTier.SEARCH_TIER_ENTERPRISE,
            search_add_ons=[vais.SearchAddOn.SEARCH_ADD_ON_LLM]
        ),
        common_config=vais.Engine.CommonConfig(company_name='acme'),
        industry_vertical=vais.IndustryVertical.GENERIC,
        data_store_ids=[DATA_STORE_ID],
    ),
    engine_id=engine_id
)
engine_client.create_engine(create_engine_request)


Note: It will take a few minutes to build the index. You may run a search request to test it.

In [ ]:
search_client = vais.SearchServiceClient(
    client_options=client_options
)
search_client.search(vais.SearchRequest(
    serving_config=(
            f'projects/{PROJECT_ID}/locations/global/collections/default_collection/engines/{engine_id}/servingConfigs/default_search'
        ),
        query='What is the 2022 Q2 revenue?',
        page_size=10,
))

## Calling Vertex LLM grounded on your Vertex AI Search engine

Once your engine is ready, you can use it as a grounding source in a Vertex LLM call, as shown below:

In [ ]:
import genai
from genai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
    grounding,
)

genai.init(project=PROJECT_ID, location=LOCATION)

model = GenerativeModel("gemini-1.5-flash-001")

tool = Tool.from_retrieval(
    grounding.Retrieval(
        grounding.VertexAISearch(
            engine=f'projects/{PROJECT_ID}/locations/global/collections/default_collection/engines/{engine_id}',
        )
    )
)

prompt = "What is the 2022 Q2 revenue?"
response = model.generate_content(
    prompt,
    tools=[tool],
    generation_config=GenerationConfig(
        temperature=0.0,
    ),
)

print(response.text)

## Cleaning up

Delete `engine` and `dataStore` created in this tutorial

In [ ]:
# Delete engine
engine_full_resource_path = f'projects/{PROJECT_ID}/locations/global/collections/default_collection/engines/{engine_id}'
delete_engine_operation = engine_client.delete_engine(engine_full_resource_path)
print(
    f"Waiting for delete engine operation to complete: {delete_engine_operation.operation.name}"
)
wait_for_operation_finish(delete_engine_operation)
print(f"Successfully deleted engine {engine_full_resource_path}")

In [ ]:
# Delete the data store
data_store_full_resource_path = f'projects/{PROJECT_ID}/locations/global/collections/default_collection/dataStores/{DATA_STORE_ID}'
delete_data_store_operation = data_store_service_client.delete_data_store(
    name=data_store_full_resource_path
)
print(
    f"Waiting for delete data store operation to complete: {delete_data_store_operation.operation.name}"
)
wait_for_operation_finish(delete_data_store_operation)
print(f"Successfully deleted data store {data_store_full_resource_path}")

In [ ]:
# Delete engine